There exist mentions of elibility requirements in page text, some in specialized `Eligibility` sections, others within the context of `smart_answers` or `answer` document_types. With a focus on age requirements, extract stuff out.

In [44]:
import os
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from pprint import pprint
import spacy
from lxml import etree
from bs4 import BeautifulSoup

In [25]:
DATA_DIR = os.getenv("DATA_DIR")
content_path = os.path.join(DATA_DIR, 
                            "preprocessed_content_store_wdetails_june_en_june.csv.gz")
df = pd.read_csv(content_path, compression="gzip")

/Users/felisialoukou/.virtualenvs/k-graph/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
df.head()

,base_path,content_id,title,description,document_type,details,orgs_id,orgs_title,sbs_details,pages_part_of_step_nav,text,taxons,locale
0,/aaib-reports/aaib-investigation-to-hawker-sea...,96eacfbe-0385-45ef-9289-8428dacad258,"AAIB investigation to Hawker Sea Fury T MK 20,...","Engine failure and landing gear collapse, RNAS...",aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The aircraft was performing in a publ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
1,/aaib-reports/aaib-investigation-to-hph-glasfl...,1d697c99-b1d0-4855-b72d-a97d83a4fc91,"AAIB investigation to HPH Glasflugel 304 eS, G...",Front Electric Sustainer (FES) battery fire du...,aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: During a normal touchdown following a...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
2,/aaib-reports/aaib-investigation-to-ikarus-c42...,5814334a-77d0-426e-8e78-ff2b05ea6322,"AAIB investigation to Ikarus C42 FB UK, G-IKUS\t",Aircraft crashed whilst avoiding a hedge when ...,aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The pilot was attempting to take off ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
3,/aaib-reports/aaib-investigation-to-ikarus-c42...,c8f31c76-eab0-4be6-95a5-5e6e7f32056c,"AAIB investigation to Ikarus C42 FB100, G-CEHG\t","Overturned on landing, Farm Strip, Hardwicke, ...",aaib_report,"{'body': [{'content_type': 'text/govspeak', 'c...",{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: G-CEHG was landing on a private airst...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en
4,/aaib-reports/aaib-investigation-to-ikarus-c42...,4cc0ae15-ad87-42ce-8a89-2bdb14e19e26,"AAIB investigation to Ikarus C42 FB100, G-ZAVI","Aircraft struck sheep on landing, Lundy Island...",aaib_report,{'metadata': {'date_of_occurrence': '2014-06-2...,{'organisations': ['38eb5d8f-2d89-480c-8655-e2...,{'organisations': ['Air Accidents Investigatio...,NaN,NaN,Summary: The pilot reported that this was his ...,"[{'title': 'Transport', 'content_id': 'a4038b2...",en


In [27]:
df['details'] = df['details'].map(literal_eval)

In [28]:
def flatten_dict(parts_dict):
    return {item['slug']:"".join([b['content'] 
                                  for b in item['body']]) for item in parts_dict}    

In [29]:
df['details_parts'] = df['details'].map(lambda x: flatten_dict(x['parts']) 
                                        if 'parts' in x.keys() 
                                          else np.nan)

In [30]:
df['eligibility_html'] = df['details_parts'].map(lambda x: x.get('eligibility',np.nan) 
                                                 if not isinstance(x,float) else np.nan)

In [31]:
df_welig = df[~df['eligibility_html'].isna()]

In [37]:
def extract_text(body):
    """
    Extract text from html body
    :param body: <str> containing html.
    """
    # TODO: Tidy this up!
    r = None
    # body != "\n" and
    if body and body != "\n" and not body.isspace():
        try:
            # print("this is", body)
            tree = etree.HTML(body)
            r = tree.xpath('//text()')
            r = ' '.join(r)
            r = r.strip().replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
            r = r.replace('\n', ' ').replace(',', ' ')
            # r = r.lower()
            r = ' '.join(r.split())
        except ValueError:
            print("exception @ extract:", type(body), body)
    if not r:
        r = ' '
    return r

In [55]:
df_welig.base_path.values

array(['/1619-bursary-fund', '/additional-state-pension',
       '/access-to-elected-office-fund', '/access-to-work',
       '/attendance-allowance', '/become-an-mot-tester',
       '/bereavement-allowance', '/bereavement-payment',
       '/bereavement-support-payment', '/blind-persons-allowance',
       '/agricultural-sick-pay', '/budgeting-help-benefits',
       '/asylum-support', '/adult-dependants-grant',
       '/adoption-pay-leave', '/advanced-learner-loan', '/ancestry-visa',
       '/apply-citizenship-born-uk', '/apply-citizenship-british-parent',
       '/apply-for-a-uk-residence-card', '/care-to-learn',
       '/carers-allowance', '/carers-credit', '/childcare-grant',
       '/christmas-bonus', '/claim-asylum',
       '/claim-compensation-criminal-injury', '/child-benefit',
       '/child-maintenance', '/child-study-visa',
       '/claim-for-injury-received-while-serving',
       '/cold-weather-payment', '/constant-attendance-allowance',
       '/derivative-right-residence', '

In [46]:
### BeautifulSoup attempt to get all the bulleted lists/headers out

In [62]:
test = df_welig.iloc[0]
text = test.eligibility_html
soup = BeautifulSoup(text, "html.parser")

In [63]:
test.base_path

'/1619-bursary-fund'

In [64]:
text

'You must:\r\n\r\n+ be at least 16 and under 19 on 31 August 2019\r\n+ study at a publicly funded school or college, or be on an unpaid training course  \r\n+ meet the residency requirements - your school or college can check this\r\n\r\n##Bursary for students in vulnerable groups \r\n\r\nYou could get up to £1,200 if at least one of the following applies: \r\n\r\n+ you\'re in or recently left local authority care\r\n+ you get Income Support or Universal Credit because you\'re financially supporting yourself \r\n+ you get Disability Living Allowance (DLA) in your name and either Employment and Support Allowance (ESA) or Universal Credit\r\n+ you get Personal Independence Payment (PIP) in your name and either ESA or Universal Credit\r\n\r\nYou may get the full amount if you have expenses and study full-time on a course of at least 30 weeks.\r\n\r\nYou’ll usually get less than the full amount, or no bursary, if one of the following apply:\r\n\r\n+ your course is shorter than 30 weeks\r\n

    if header -> new scenario
    first header (overall?)
    other headers -> secondary scenarios, unless first header mentions a requirement disputed by scenario, then it's a different class

In [108]:
def specialized_scenario(header_string):
    return any([x in header_string for x in [' for ','group']]) 

In [109]:
specialized_scenario("group")

True

In [275]:
class Section:
    
    def __init__(self, section_type="", header=""):
        self.section_type = section_type
        self.header = header
        self.bullets = []
        self.paragraphs = []
    
    def set_type(self, section_type):
        self.section_type = section_type
        
    def set_header(self, header):
        self.header = header
        
    def add_bullet(self, bullet):
        if bullet not in self.bullets:
            self.bullets.append(bullet)
            
    def add_paragraph(self, paragraph):
        if paragraph not in self.paragraphs:
            self.paragraphs.append(paragraph)   
            
    def to_string(self):
        return "type: {}\nheader: {}\nbullets: {}\nparagraphs: {}"\
                    .format(self.section_type, self.header, self.bullets, self.paragraphs)
            

In [276]:
section = Section()
section.add_bullet("womp")
section.section_type
section.bullets

['womp']

In [277]:
            
#         if header.name = "ul":
# #             print(header.name, header.text.strip())
#         elif header.name == "ul":
#             for bullet in header.find_all(["li"]):
# #                 print("+", bullet.text.strip())
#         else:
#             print("bonk")
#         if "h" in header.name and specialized_scenario(header.text.strip()):
#             print('https://www.gov.uk'+row['base_path']+'/eligibility')
#             print("secondary case?")
#             print(header.name, header.text.strip())
#         print("###")

In [278]:
df_welig[0:2]

,base_path,content_id,title,description,document_type,details,orgs_id,orgs_title,sbs_details,pages_part_of_step_nav,text,taxons,locale,details_parts,eligibility_html
20,/1619-bursary-fund,f4b96a38-5247-4afd-b554-8a258a0e8c93,16 to 19 Bursary Fund,"Bursaries of up to £1,200 for students in furt...",guide,"{'parts': [{'title': 'Overview', 'slug': 'over...",{'organisations': ['71381a6e-aa5c-43ae-a982-be...,{'organisations': ['Education and Skills Fundi...,NaN,NaN,You could get a bursary to help with education...,"[{'title': 'Education, training and skills', '...",en,{'overview': 'You could get a bursary to help ...,You must:\r\n\r\n+ be at least 16 and under 19...
8732,/additional-state-pension,e78637eb-3be4-408c-9f9c-d2336635c0ca,Additional State Pension,"Additional State Pension, also known as the St...",guide,"{'parts': [{'title': 'Overview', 'slug': 'over...",{'organisations': ['b548a09f-8b35-4104-89f4-f1...,{'organisations': ['Department for Work and Pe...,NaN,NaN,The Additional State Pension is an extra amoun...,"[{'title': 'Work', 'content_id': 'd0f1e5a3-c8f...",en,{'overview': 'The Additional State Pension is ...,##You reached State Pension age on or after 6 ...


In [280]:
all_stacks = []
for i,row in df_welig[0:2].iterrows():
    text = row['eligibility_html']
    soup = BeautifulSoup(text, "html.parser")
    section_stack = []
    for i, header in enumerate(soup.find_all(["h1","h2","h3", "p", "ul"])):
        print(i)
        print(header.text)
        # first element on eligibility page
        if i == 0:
            section_stack = [Section()]
            ## Page is sectioned by default, multiple scenarios
            if header.name.startswith("h"):
                section_stack[-1].set_type("secondary")
                
            ## Page has a "main" scenario, may include secondary scenarios 
            ## (that may not abide to stuff outlined in main scenario)
            if header.name == "p":
                section_stack[-1].set_type("base")   
                
            section_stack[-1].set_header(header.text.strip())
        else:
            if header.name.startswith("h"):
                section_stack.append(Section("secondary",header.text.strip()))
            ## Page has a "main" scenario, may include secondary scenarios 
            ## (that may not abide to stuff outlined in main scenario)
            if header.name == "p":
                section_stack[-1].add_paragraph(header.text.strip())
            else:
                for bullet in header.find_all(["li"]):
                    section_stack[-1].add_bullet(bullet.text.strip())
    all_stacks.append((row['base_path'],section_stack))

0
You must:
1

be at least 16 and under 19 on 31 August 2019
study at a publicly funded school or college, or be on an unpaid training course
meet the residency requirements - your school or college can check this

2
Bursary for students in vulnerable groups
3
You could get up to £1,200 if at least one of the following applies:
4

you’re in or recently left local authority care
you get Income Support or Universal Credit because you’re financially supporting yourself
you get Disability Living Allowance (DLA) in your name and either Employment and Support Allowance (ESA) or Universal Credit
you get Personal Independence Payment (PIP) in your name and either ESA or Universal Credit

5
You may get the full amount if you have expenses and study full-time on a course of at least 30 weeks.
6
You’ll usually get less than the full amount, or no bursary, if one of the following apply:
7

your course is shorter than 30 weeks
you study part time
you have few expenses

8
You’ll be told what evidenc

In [281]:
for section in all_stacks[0][1]:
    print(section.to_string())
    print("###")

type: base
header: You must:
bullets: ['be at least 16 and under 19 on 31 August 2019', 'study at a publicly funded school or college, or be on an unpaid training course', 'meet the residency requirements - your school or college can check this']
paragraphs: []
###
type: secondary
header: Bursary for students in vulnerable groups
bullets: ['you’re in or recently left local authority care', 'you get Income Support or Universal Credit because you’re financially supporting yourself', 'you get Disability Living Allowance (DLA) in your name and either Employment and Support Allowance (ESA) or Universal Credit', 'you get Personal Independence Payment (PIP) in your name and either ESA or Universal Credit', 'your course is shorter than 30 weeks', 'you study part time', 'you have few expenses']
paragraphs: ['You could get up to £1,200 if at least one of the following applies:', 'You may get the full amount if you have expenses and study full-time on a course of at least 30 weeks.', 'You’ll usua

In [261]:
### Regex stuff

In [ ]:
rowlist = []
for key,value in list(elig_texts.items()):

    if "This fund is currently closed." not in value:
#         \s?\d\d\s?(and|or)?\s?(over|under)?(\s\d\d)?
        search = re.finditer(""".*(born on|(reached|over|under) State Pension age|
                             age(d)?|('re|were|are|be( at least)?|have been)\s?(over|under)?\d\d).*""", value)
        if search is not None:
#             print(f'https://www.gov.uk{key}/eligibility')
            for i,s in enumerate(search):
                start = s.start()
                end = s.end()
#                 print(f'At {i}: ({start},{end})')
                text = value[start:end]
#                 print(text)
                rowlist.append({'base_path':key, 
                                'instance #':int(i+1),
                                'start':start,
                                'end':end,
                                'age_text':value[start:end],
                                'text':value})
        else:
            rowlist.append({'base_path':key, 'age_text':'Age requirements not detected',
                           'text':value})
            
    else:
#         print(f'https://www.gov.uk{key}/eligibility')
#         print("closed")
        rowlist.append({'base_path':key, 'age_text':'Fund closed','text':value})
df_eligibility = pd.DataFrame(rowlist)
df_eligibility = df_eligibility[['base_path', 'text','instance #', 'start', 'end','age_text']]